In [2]:
#%%capture output

# once finished, try on all HAPI records and export output to file by doing the top and bottom comments

from datetime import datetime, timedelta
import Scripts
from Scripts import create_sqlite_database, execution, executionALL
from hapiclient import hapi
from hapiclient.util import error
from time import sleep
import multiprocessing

# check and see if data can be retrieved from HAPI server
def DataChecker(server, dataset, start, stop, parameters, return_dict):
    # control loops
    dataFound = False
    broken = False
    
    try:
        #sleep(5.0)
        data, meta = hapi(server, dataset, parameters, start, stop)

    # if error(s) arise
    #except HAPIError as err:
    except Exception as err:
        #print(err)
        broken = True
        #pass

    # if no error arises
    else:
        # search for data
        while not dataFound:
            if len(data) == 0:
                pass
            else:
                #for entry in data:                
                # once data is found, end loop
                if str(data[0][0]) != "":
                    dataFound = True
                    print("Data was successfully accessed")
                    #print("Example data looks like " + str(data[0][0]))
    finally:
        return_dict["dataFound"] = dataFound
        return_dict["attempts"] += 1
        return_dict["broken"] = broken
        return None

# retrieve prodKeys associated with HAPI URLs

# input abs path of database file you wish to query from
conn = create_sqlite_database("/home/jovyan/HDRL-Internship-2024/SPASE_Data_20240716.db")

# fails data checks 
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 7"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 21"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 59"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 5 OFFSET 64"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 130"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 136"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 283""" takes a few tries

# examples that are not actual datasets according to CDAWEB
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 3 OFFSET 120"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 14 OFFSET 233"""


# example w multiple prodKeys
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 45"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 67"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 93"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 96"""

# example where it passes data check at another interval besides 1sec
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 23"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 248""" incorrect
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 4 OFFSET 258""" incorrect

HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 10 OFFSET 285"""


"""spase://NASA/NumericalData/AMPTE-CCE/MEPA/PT0.1875S
spase://NASA/NumericalData/DE2/IDM/PT0.25S
spase://NASA/NumericalData/FAST/MAG/Fluxgate/PT7.8125MS
spase://NASA/NumericalData/ISIS1/SFS/Ionogram/PT29S
spase://NASA/NumericalData/MESSENGER/MAG/PT0.05S"""

prodKeys = []
prodKeys = execution(HapiStmt, conn)
print("The datasets are " + str(prodKeys))


server = 'https://cdaweb.gsfc.nasa.gov/hapi'

# iterate thru prodKeys to assign as dataset
for prodKey in prodKeys:
    # check if multiple prodKeys for same URL (mult keys in one string)
    if ", " in prodKey:
        print("This HAPI URL has multiple product keys.")
        index = prodKeys.index(prodKey)
        prodKey = prodKey.replace("\'", "")
        # keep separating them until each prodKey is in own string
        # while ", " in prodKey:
        before, sep, after = prodKey.partition(", ")
        prodKeys[index] = before # remove this line if need to check all keys
        #prodKeys[index] = after
        #prodKeys.insert(index, before)
        prodKey = prodKeys[index]
    dataset = str(prodKey)
    
    # control loops
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    # count of attempts to get data
    return_dict["attempts"] = 0
    return_dict["dataFound"] = False
    return_dict["broken"] = False
    #dataFound = False
    #broken = False
    
    # list that holds all parameters for a given server
    paramNames =  []

    # retrieve all parameters and the start date from the server
    sleep(5.0)
    meta = hapi(server,dataset)
    # get parameters
    for k, v in meta.items():
        if k == "parameters":
            for params in v:
                for key, value in params.items():
                    if key == "name":
                        paramNames.append(value)
        # get start date
        elif k == "startDate":
            start = v

    #print(paramNames)

    # dictionary that holds datetime obj for each interval
    intervals = {}
    intervals["1s"] = ""
    intervals["10s"] = ""
    intervals["1min"] = ""
    intervals["10min"] = ""
    intervals["1hr"] = ""
    intervals["1d"] = ""
    intervals["3d"] = ""
    intervals["1w"] = ""
    intervals["1mon"] = ""
    
    # create incremental intervals to test for data check
    date, sep, time = start.partition("T")
    time = time.replace("Z", "")
    dt_string = date + " " + time
    dt_obj = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
    # 1 second
    intervals["1s"] = dt_obj + timedelta(seconds=1)
    # 10 seconds
    intervals["10s"] = dt_obj + timedelta(seconds=10)
    # 1 minute
    intervals["1min"] = dt_obj + timedelta(minutes=1)
    # 10 minutes
    intervals["10min"] = dt_obj + timedelta(minutes=10)
    # hour
    intervals["1hr"] = dt_obj + timedelta(hours=1)
    # day
    intervals["1d"] = dt_obj + timedelta(days=1)
    # 3 days
    intervals["3d"] = dt_obj + timedelta(days=3)
    # week
    intervals["1w"] = dt_obj + timedelta(weeks=1)
    # month
    intervals["1mon"] = dt_obj + timedelta(weeks=4,days=2)

    # to iterate thru all parameters in a server, use the for loop below to enclose the code below
    # for parameters in paramNames[1:]:
    
    # UNFINISHED: have data check occur in increasingly larger start/stop intervals until data is returned
    #              and only check for one parameter (Time is fine)
    parameter = paramNames[0]
    for k, v in intervals.items():
        if (not return_dict["dataFound"]) and (not return_dict["broken"]):
            stop = str(v)
            stop = stop.replace(" ", "T") + "Z"
            print("Checking parameter " + str(parameter) + " in HAPI record with id " + dataset +
                  " at the interval of " + str(k))
            """dataFound, attempts, broken = DataChecker(server, dataset, start, stop,
                                              parameter, attempts)"""
            p = multiprocessing.Process(target=DataChecker, args=(server, dataset, start, stop,
                                              parameter, return_dict))
            sleep(5.0)
            p.start()
            p.join(10)
            # add export to text file that contains the prodKey/dataset that took too long for a retry later
            if p.is_alive():
                print(f"Data retrieval took more than 10 seconds for dataset at position {prodKeys.index(prodKey)}" +
                      " trying again at next interval.")
                p.terminate()
                return_dict["attempts"] += 1
    # if all intervals fail or link is broken -> no data
    if not return_dict["dataFound"]:
    # inputs "HAPI info check passed after 1 attempt. HAPI data check failed after x attempts."
    #     into "Error" column in TestResults associated with that HAPI URL
        print("No data was found")

#with open("../DatalinkCheckOutput.txt", "w") as file:
 #   file.write(output.stdout)

The datasets are ['AC_K1_MFI']
Checking parameter Time in HAPI record with id AC_K1_MFI at the interval of 1s
Data retrieval took more than 10 seconds for dataset at position 0 trying again at next interval.
Checking parameter Time in HAPI record with id AC_K1_MFI at the interval of 10s
Data retrieval took more than 10 seconds for dataset at position 0 trying again at next interval.
Checking parameter Time in HAPI record with id AC_K1_MFI at the interval of 1min
Data was successfully accessed


In [ ]:
print("The program is done!")

In [16]:
# call .py file directly from notebook
#%run ./HAPICheck.py > ../DatalinkCheckOutput.txt

In [ ]:
        errMessage = "HAPI info check passed after 1 attempt. HAPI data check"
        errMessage += f" failed after {return_dict['attempts']} attempt(s)."
        HAPIErrorStmt = f""" UPDATE TestResults
                                SET Errors = '{errMessage}'
                                FROM (SELECT SPASE_id, prodKey FROM TestResults
                                        INNER JOIN MetadataEntries USING (SPASE_id))
                                WHERE prodKey = '{dataset}' """
        Record_id = execution(f""" SELECT rowNum 
                                FROM (SELECT TestResults.rowNum, SPASE_id, prodKey FROM TestResults 
                                    INNER JOIN MetadataEntries USING (SPASE_id))
                                WHERE prodKey = '{dataset}';""", conn)
        executionALL(HAPIErrorStmt, conn)
        print(f"Sent error message to a TestResults entry with the row number {Record_id}")

In [ ]:
from hapiclient.util import HAPIError
dir(HAPIError)

In [54]:
prodKeys.index("I1_AV_ULA")

29